In [23]:
import requests, json, csv, io, os
from maniera.calculator import Maniera

import time, memory_profiler, threading # for diagnostics/performance eval

In [24]:
## Calculate converted performance points and star (difficulty) rating for a score with given modifiers.
## Parameters:
    # h   - the score to be converted
    # mod - the score's modifiers, as a string (DT, NC, HT, 0 for none)
def mod_converter(h,mod):
    # extract relevant fields for constructing a filename
    artist = h['beatmapset']['artist']
    song = h['beatmapset']['title']
    mapper = h['beatmapset']['creator']
    difficulty = h['beatmap']['version']
    
    score = h['score'];
    
    # dictionary representing reduced bitwise enum for modifiers, see https://github.com/ppy/osu-api/wiki#mods
    modEnum = {'DT': 64, 'NC': 576, 'HT': 256, 0: 0}

    # delete leading bracket fields because they are ignored in filenames
    if (difficulty[0] == '['):
        difficulty = difficulty[difficulty.find(']') + 2:]

    # produce formatted filename
    filename = 'maps\%s - %s (%s) [%s].osu' % (artist, song, mapper, difficulty)
    filename = filename.replace('/', '').replace('?', '').replace('*', '').replace('"', '') # remove illegal filename chars

    # remove non-ASCII chars by writing to a temporary .osu file
    with io.open(filename,'r',encoding='utf-8',errors='ignore') as infile, io.open('temp.osu','w',encoding='ascii',errors='ignore') as outfile:
        for line in infile:
            print(*line.split(), file=outfile)

    # calculate new star rating and pp
    calc = Maniera('temp.osu', modEnum[mod], score)
    calc.calculate()
    
    os.remove('temp.osu') # delete temporary .osu file
    
    return round(calc.pp,2), round(calc.sr,3);



In [25]:
startTime = time.time();

In [26]:
### Retrieve token

configName = 'config.json'; # location of json file containing header params (api key, etc.)

''' create config.json in this form, put in root directory
{
    'grant_type': 'client_credentials',
    'client_id': 1,
    'client_secret': "put yours here",
    'scope': 'public'
}
'''
with open(configName) as file:
    params = json.load(file);

token = requests.post('https://osu.ppy.sh/oauth/token', params).json()['access_token']

headers = {
    'Authorization': 'Bearer ' + token
}

In [27]:
# id = 1815478 #me
# id = 259972 #jakads
id = 5610085 #etiennexc

url = 'https://osu.ppy.sh/api/v2/users/{}/scores/best?limit=51'.format(id) # api endpoint url

r = requests.get(url, headers = headers)

## Write data to .csv
stats = []
for h in r.json():
    if h['mods'] == []:
        temp2 = {'mods':0}
    else:
        if any(x == 'DT' for x in h['mods']) or any(x == 'NC' for x in h['mods']):
            temp2 = {'mods':'DT'}
            h['pp'], h['beatmap']['difficulty_rating'] = mod_converter(h,temp2['mods'])
            h['beatmap']['bpm'] *= 1.5
        elif any(x == 'HT' for x in h['mods']):
            temp2 = {'mods':'HT'}
            h['pp'], h['beatmap']['difficulty_rating'] = mod_converter(h,temp2['mods'])
            h['beatmap']['bpm'] *= .75
    temp3 = {'pp':h['pp']}
    temp = h['beatmap'].copy()
    temp.update(h['statistics'])
    temp.update(temp2)
    temp.update(temp3)
    for e in ['id','mode','version','ar','beatmapset_id','convert','count_spinners','cs','deleted_at','drain','is_scoreable','last_updated','mode_int','ranked','status','url']:
        temp.pop(e)
    stats.append(temp)
keys = stats[0].keys()
filename = '{}.csv'.format(id)
with open(filename, 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(stats)

In [28]:
stats

[{'difficulty_rating': 7.309,
  'accuracy': 8.5,
  'bpm': 262.5,
  'count_circles': 3818,
  'count_sliders': 35,
  'hit_length': 319,
  'passcount': 184316,
  'playcount': 698531,
  'total_length': 319,
  'count_50': 0,
  'count_100': 7,
  'count_300': 640,
  'count_geki': 3156,
  'count_katu': 42,
  'count_miss': 8,
  'mods': 'DT',
  'pp': 609.48},
 {'difficulty_rating': 7.477,
  'accuracy': 9,
  'bpm': 120.0,
  'count_circles': 1764,
  'count_sliders': 7,
  'hit_length': 123,
  'passcount': 23361,
  'playcount': 111880,
  'total_length': 126,
  'count_50': 5,
  'count_100': 23,
  'count_300': 404,
  'count_geki': 1209,
  'count_katu': 113,
  'count_miss': 17,
  'mods': 'DT',
  'pp': 520.94},
 {'difficulty_rating': 7.01,
  'accuracy': 9,
  'bpm': 256,
  'count_circles': 2404,
  'count_sliders': 57,
  'hit_length': 126,
  'passcount': 59899,
  'playcount': 410658,
  'total_length': 127,
  'count_50': 2,
  'count_100': 4,
  'count_300': 556,
  'count_geki': 1833,
  'count_katu': 64,
  '

In [29]:
print("Time to execute: %.4f seconds" % (time.time() - startTime))

Time to execute: 1.7473 seconds
